
# ChatGPT Prompt Engineering For Developers

## Unit 1: Introduction

For developers, the power of LLM (Large Language Models) comes in the ability to deliver rich experiences by making API calls to an LLM to achieve tasks like summarization, inference, transformation, and expansion - of text inputs. In this course, we learn to build a chat bot using API calls to LLM.

There are 2 types of LLM: _Base LLM_ trained to predict the next word, and _Instruction-Tuned LLM_ trained to follow instructions. The latter uses Reinforcement Learning with Human Feedback (RLHF) and is trained to be _Helpful, Honest, Harmless_. This course recommends we prioritize focus on Instruction-Tuned LLM (IT-LLM) and teaches best practices for doing so.

Think of an IT-LLM as a smart-friend but not a mind reader. In other words, when an IT-LLM fails to deliver an expected result, it might be because the _instructions it was given_ were not representative. Some guidance:
 - Be Specific: Was I clear in what I wanted the focus of the result to be?
 - Set Tone: Was I clear about the tone (conversational, formal, social etc.) to use in writing?
 - Give Context: Can I provide explicit snippets of text or references that can help specialize this further?

References:
1. [OpenAI Cookbook: Recipes](https://github.com/openai/openai-cookbook)
2. [OpenAI Docs: Tutorials](https://platform.openai.com/docs/tutorials)
3. [OpenAI Gallery: Examples](https://platform.openai.com/examples)

## Unit 2: Guidelines for Prompting

2 Principles for Prompt Writing:
 1. Write clear and specific prompts.
 2. Give the model time to think.

### Setup for OpenAI Exercises

Adding this section to document how we configure our Codespaces to support this exercises:
1. Install `openai` Python library. 
    - Use `pip install openai` to do this temporarily, in current container
    - Update _requirements.txt_ to add `openai` to do this permanently (and rebuild current container).
2. Set the OPENAI_API_KEY environment variable to your OpenAI key.
    - First, get an OpenAI account. Create new secret key [under API keys](https://platform.openai.com/account/api-keys) - and copy it.
    - For Codespaces, [create a Codespaces secret](https://github.com/settings/codespaces) with name OPENAI_API_KEY
    - Paste the OpenAI secret key you copied previously into the _Value_ field of the Codespaces secret.
    
    - Select repositories that should be given access to this key, and "Add secret"
    - The secret is now _automatically exported_ [as an env var in user's terminal](https://docs.github.com/en/codespaces/managing-your-codespaces/managing-encrypted-secrets-for-your-codespaces#using-secrets)
    - Rebuild container to enforce new secrets - test in terminal with `echo $OPENAI_API_KEY`